# COMS W4705 FALL 24
## Homework 4 - Semantic Role Labelling with BERT

The goal of this assignment is to train and evaluate a PropBank-style semantic role labeling (SRL) system. Following (Collobert et al. 2011) and others, we will treat this problem as a sequence-labeling task. For each input token, the system will predict a B-I-O tag, as illustrated in the following example:

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|||schedule.01|||||||||||||

Note that the same sentence may have multiple annotations for different predicates

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
|B-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|I-ARG1|O|B-V|B-ARG2|I-ARG2|I-ARG2|B-ARGM-TMP|O|
||||||||||remove.01||||||

and not all predicates need to be verbs

|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|    
|O|O|O|O|O|O|B-ARG1|B-V|O|O|O|O|O|O|O|
||||||||try.02||||||||

The SRL system will be implemented in [PyTorch](https://pytorch.org/). We will use BERT (in the implementation provided by the [Huggingface transformers](https://huggingface.co/docs/transformers/index) library) to compute contextualized token representations and a custom classification head to predict semantic roles. We will fine-tune the pretrained BERT model on the SRL task.


### Overview of the Approach

The model we will train is pretty straightforward. Essentially, we will just encode the sentence with BERT, then take the contextualized embedding for each token and feed it into a classifier to predict the corresponding tag.

Because we are only working on argument identification and labeling (not predicate identification), it is essentially that we tell the model where the predicate is. This can be accomplished in various ways. The approach we will choose here repurposes Bert's *segment embeddings*.

Recall that BERT is trained on two input sentences, seperated by [SEP], and on a next-sentence-prediction objective (in addition to the masked LM objective). To help BERT comprehend which sentence a given token belongs to, the original BERT uses a segment embedding, using A for the first sentene, and B for the second sentence 2.
Because we are labeling only a single sentence at a time, we can use the segment embeddings to indicate the predicate position instead: The predicate is labeled as segment B (1) and all other tokens will be labeled as segment A (0).

<img src="https://github.com/daniel-bauer/4705-f23-hw5/blob/main/bert_srl_model.png?raw=true" width=400px>

## Setup: GCP, Jupyter, PyTorch, GPU

To make sure that PyTorch is available and can use the GPU,run the following cell which should return True. If it doesn't, make sure the GPU drivers and CUDA are installed correctly.

GPU support is required for this assignment -- you will not be able to fine-tune BERT on a CPU.

In [1]:
import torch
torch.cuda.is_available()

True

## Dataset: Ontonotes 5.0 English SRL annotations

We will work with the English part of the [Ontonotes 5.0](https://catalog.ldc.upenn.edu/LDC2013T19) data. This is an extension of PropBank, using the same type of annotation. Ontonotes contains annotations other than predicate/argument structures, but we will use the PropBank style SRL annotations only. *Important*: This data set is provided to you for use in COMS 4705 only! Columbia is a subscriber to LDC and is allowed to use the data for educational purposes. However, you may not use the dataset in projects unrelated to Columbia teaching or research.

If you haven't done so already, you can download the data here:


In [ ]:
! wget https://storage.googleapis.com/4705-bert-srl-data/ontonotes_srl.zip

In [ ]:
! unzip ontonotes_srl.zip

The data has been pre-processed in the following format. There are three files:

`propbank_dev.tsv`	`propbank_test.tsv`	`propbank_train.tsv`

Each of these files is in a tab-separated value format. A single predicate/argument structure annotation consists of four rows. For example

```
ontonotes/bc/cnn/00/cnn_0000.152.1
The     judge   scheduled       to      preside over    his     trial   was     removed from    the     case    today   /.
                schedule.01
B-ARG1  I-ARG1  B-V     B-ARG2  I-ARG2  I-ARG2  I-ARG2  I-ARG2  O       O       O       O       O       O       O
```

* The first row is a unique identifier (1st annotation of the 152nd sentence in the file ontonotes/bc/cnn/00/cnn_0000).
* The second row contains the tokens of the sentence (tab-separated).
* The third row contains the probank frame name for the predicate (empty field for all other tokens).
* The fourth row contains the B-I-O tag for each token.

The file `rolelist.txt` contains a list of propbank BIO labels in the dataset (i.e. possible output tokens). This list has been filtered to contain only roles that appeared more than 1000 times in the training data.
We will load this list and create mappings from numeric ids to BIO tags and back.

In [2]:
role_to_id = {}
with open("role_list.txt",'r') as f:
    role_list = [x.strip() for x in f.readlines()]
    role_to_id = dict((role, index) for (index, role) in enumerate(role_list))
    role_to_id['[PAD]'] = -100

    id_to_role = dict((index, role) for (role, index) in role_to_id.items())


Note that we are also mapping the '[PAD]' token to the value -100. This allows the loss function to ignore these tokens during training.

## Part 1 - Data Preparation

Before you can build the SRL model, you first need to preprocess the data.


### 1.1 - Tokenization

One challenge is that the pre-trained BERT model uses subword ("WordPiece") tokenization, but the Ontonotes data does not. Fortunately Huggingface transformers provides a tokenizer.

In [3]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.tokenize("This is an unbelievably boring test sentence.")

['this',
 'is',
 'an',
 'un',
 '##bel',
 '##ie',
 '##va',
 '##bly',
 'boring',
 'test',
 'sentence',
 '.']

**TODO**:
We need to be able to maintain the correct labels (B-I-O tags) for each of the subwords.
Complete the following function that takes a list of tokens and a list of B-I-O labels of the same length as parameters, and returns a new token / label pair, as illustrated in the following example.


```
>>> tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)
(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

```

To approach this problem, iterate through each word/label pair in the sentence. Call the tokenizer on the word. This may result in one or more tokens. Create the correct number of labels to match the number of tokens. Take care to not generate multiple B- tokens.


This approach is a bit slower than tokenizing the entire sentence, but is necessary to produce proper input tokenization for the pre-trained BERT model, and the matching target labels.

In [4]:
def tokenize_with_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces.
    """
    tokenized_sentence = []
    labels = []
    for word, label in zip(sentence, text_labels):
        tokens = tokenizer.tokenize(word)
        label = label if label in role_to_id else 'O'
        tokenized_sentence.extend(tokens)
        labels.extend([label] + ['I-' + label[2:] if label.startswith('B-') else label] * (len(tokens) - 1))
    return tokenized_sentence, labels

In [7]:
tokenize_with_labels("the fancyful penguin devoured yummy fish .".split(), "B-ARG0 I-ARG0 I-ARG0 B-V B-ARG1 I-ARG1 O".split(), tokenizer)

(['the',
  'fancy',
  '##ful',
  'penguin',
  'dev',
  '##oured',
  'yu',
  '##mmy',
  'fish',
  '.'],
 ['B-ARG0',
  'I-ARG0',
  'I-ARG0',
  'I-ARG0',
  'B-V',
  'I-V',
  'B-ARG1',
  'I-ARG1',
  'I-ARG1',
  'O'])

### 1.2 Loading the Dataset

Next, we are creating a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class. This class acts as a contained for the training, development, and testing data in memory. You should already be familiar with Datasets and Dataloaders from homework 3.

1.2.1 **TODO**: Write the \_\_init\_\_(self, filename) method that reads in the data from a data file (specified by the filename).

For each annotation you start with  the tokens in the sentence, and the BIO tags. Then you need to create the following

1. call the `tokenize_with_labels` function to tokenize the sentence.
2. Add the (token, label) pair to the self.items list.

1.2.2 **TODO**: Write the \_\_len\_\_(self) method that returns the total number of items.

1.2.3 **TODO**: Write the \_\_getitem\_\_(self, k) method that returns a single item in a format BERT will understand.
* We need to process the sentence by adding "\[CLS\]" as the first token and "\[SEP\]" as the last token. The need to pad the token sequence to 128 tokens using the "\[PAD\]" symbol. This needs to happen both for the inputs (sentence token sequence) and outputs (BIO tag sequence).
* We need to create an *attention mask*, which is a sequence of 128 tokens indicating the actual input symbols (as a 1) and \[PAD\] symbols (as a 0).
* We need to create a *predicate indicator* mask, which is a sequence of 128 tokens with at most one 1, in the position of the "B-V" tag. All other entries should be 0. The model will use this information to understand where the predicate is located.

* Finally, we need to convert the token and tag sequence into numeric indices. For the tokens, this can be done using the `tokenizer.convert_tokens_to_ids` method. For the tags, use the `role_to_id` dictionary.
Each sequence must be a pytorch tensor of shape (1,128). You can convert a list of integer values like this `torch.tensor(token_ids, dtype=torch.long)`.

To keep everything organized, we will return a dictionary in the following format

```
{'ids': token_tensor,
 'targets': tag_tensor,
 'mask': attention_mask_tensor,
 'pred': predicate_indicator_tensor}
```


(Hint: To debug these, read in the first annotation only / the first few annotations)


In [5]:
from torch.utils.data import Dataset, DataLoader
class SrlData(Dataset):
    def __init__(self, filename):
        super(SrlData, self).__init__()

        self.max_len = 128
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.items = []

        role_to_id = {}
        with open("role_list.txt", 'r') as f:
            role_list = [x.strip() for x in f.readlines()]
            role_to_id = {role: index for index, role in enumerate(role_list)}
            role_to_id['[PAD]'] = -100

        with open(filename, 'r') as file:
            lines = file.readlines()

        for i in range(0, len(lines), 4):
            sentence = lines[i + 1].strip().split('\t')
            text_labels = lines[i + 3].strip().split('\t')
            tokenized_sentence, labels = tokenize_with_labels(sentence, text_labels, self.tokenizer)
            labels = [label if label in role_to_id else "O" for label in labels]
            self.items.append((tokenized_sentence, labels, role_to_id))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, k):
        tokens, labels, role_to_id = self.items[k]
        tokens = ["[CLS]"] + tokens[:self.max_len - 2] + ["[SEP]"]
        labels = ["[PAD]"] + labels[:self.max_len - 2] + ["[PAD]"]
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        label_ids = [role_to_id[label] for label in labels]
        attention_mask = [1] * len(token_ids)
        pred_indicator = [1 if label == "B-V" else 0 for label in labels]
        padding_length = self.max_len - len(token_ids)
        token_ids += [self.tokenizer.pad_token_id] * padding_length
        label_ids += [-100] * padding_length
        attention_mask += [0] * padding_length
        pred_indicator += [0] * padding_length
        return {
            'ids': torch.tensor(token_ids, dtype=torch.long),
            'targets': torch.tensor(label_ids, dtype=torch.long),
            'mask': torch.tensor(attention_mask, dtype=torch.long),
            'pred': torch.tensor(pred_indicator, dtype=torch.long)
        }

In [6]:
# Reading the training data takes a while for the entire data because we preprocess all data offline
data = SrlData("propbank_train.tsv")

## 2. Model Definition

In [7]:
from torch.nn import Module, Linear, CrossEntropyLoss
from transformers import BertModel

We will define the pyTorch model as a subclass of the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class. The code for the model is provided for you. It may help to take a look at the documentation to remind you of how Module works. Take a look at how the huggingface BERT model simply becomes another sub-module.

In [8]:
class SrlModel(Module):

    def __init__(self):

        super(SrlModel, self).__init__()

        self.encoder = BertModel.from_pretrained("bert-base-uncased")

        # The following two lines would freeze the BERT parameters and allow us to train the classifier by itself.
        # We are fine-tuning the model, so you can leave this commented out!
        # for param in self.encoder.parameters():
        #    param.requires_grad = False

        # The linear classifier head, see model figure in the introduction.
        self.classifier = Linear(768, len(role_to_id))


    def forward(self, input_ids, attn_mask, pred_indicator):

        # This defines the flow of data through the model

        # Note the use of the "token type ids" which represents the segment encoding explained in the introduction.
        # In our segment encoding, 1 indicates the predicate, and 0 indicates everything else.
        bert_output =  self.encoder(input_ids=input_ids, attention_mask=attn_mask, token_type_ids=pred_indicator)

        enc_tokens = bert_output[0] # the result of encoding the input with BERT
        logits = self.classifier(enc_tokens) #feed into the classification layer to produce scores for each tag.

        # Note that we are only interested in the argmax for each token, so we do not have to normalize
        # to a probability distribution using softmax. The CrossEntropyLoss loss function takes this into account.
        # It essentially computes the softmax first and then computes the negative log-likelihood for the target classes.
        return logits

In [9]:
model = SrlModel().to('cuda') # create new model and store weights in GPU memory

Now we are ready to try running the model with just a single input example to check if it is working correctly. Clearly it has not been trained, so the output is not what we expect. But we can see what the loss looks like for an initial sanity check.

**TODO**:
* Take a single data item from the dev set, as provided by your Dataset class defined above. Obtain the input token ids, attention mask, predicate indicator mask, and target labels.
* Run the model on the ids, attention mask, and predicate mask like this:

In [10]:
# pick an item from the dataset. Then run
dev_dataset = SrlData("propbank_dev.tsv")
sample = dev_dataset[0]
ids = sample['ids'].unsqueeze(0).cuda()
mask = sample['mask'].unsqueeze(0).cuda()
pred = sample['pred'].unsqueeze(0).cuda()
targets = sample['targets'].unsqueeze(0).cuda()
outputs = model(ids, mask, pred)
print(outputs)
print('')
print(outputs.shape)

tensor([[[ 0.3652, -0.5642, -0.2287,  ...,  0.0164, -0.1385, -0.4647],
         [ 0.2238, -0.2687, -0.1406,  ...,  0.7741, -0.1270, -0.4423],
         [ 0.4599, -0.3520, -0.1260,  ...,  0.3172, -0.2273, -0.2260],
         ...,
         [ 0.1316,  0.0184, -0.2946,  ...,  0.2172,  0.1210,  0.1414],
         [ 0.2737, -0.2816, -0.2754,  ...,  0.1544,  0.0884,  0.1335],
         [ 0.0914, -0.0276, -0.2341,  ...,  0.2346,  0.0903,  0.2031]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

torch.Size([1, 128, 53])


**TODO**:
Compute the loss on this one item only.
The initial loss should be close to -ln(1/num_labels)

Without training we would assume that all labels for each token (including the target label) are equally likely, so the negative log probability for the targets should be approximately $$-\ln(\frac{1}{\text{num_labels}}).$$ This is what the loss function should return on a single example. This is a good sanity check to run for any multi-class prediction problem.

In [11]:
import math
-math.log(1 / len(role_to_id), math.e)

3.970291913552122

In [12]:
loss_function = CrossEntropyLoss(ignore_index = -100, reduction='mean')

# complete this. Note that you still have to provide a (batch_size, input_pos)
# tensor for each parameter, where batch_size =1
reshaped_outputs = outputs.view(-1, len(role_to_id))
reshaped_targets = targets.view(-1)
loss = loss_function(reshaped_outputs, reshaped_targets)
print(loss.item())
# outputs = model(ids, mask, pred)
# loss = loss_function(...)
# loss.item()   #this should be approximately the score from the previous cell


4.283106327056885


**TODO**: At this point you should also obtain the actual predictions by taking the argmax over each position.
The result should look something like this (values will differ).

```
tensor([[ 1,  4,  4,  4,  4,  4,  5, 29, 29, 29,  4, 28,  6, 32, 32, 32, 32, 32,
         32, 32, 30, 30, 32, 30, 32,  4, 32, 32, 30,  4, 49,  4, 49, 32, 30,  4,
         32,  4, 32, 32,  4,  2,  4,  4, 32,  4, 32, 32, 32, 32, 30, 32, 32, 30,
         32,  4,  4, 49,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  6,  6, 32, 32,
         30, 32, 32, 32, 32, 32, 30, 30, 30, 32, 30, 49, 49, 32, 32, 30,  4,  4,
          4,  4, 29,  4,  4,  4,  4,  4,  4, 32,  4,  4,  4, 32,  4, 30,  4, 32,
         30,  4, 32,  4,  4,  4,  4,  4, 32,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4]], device='cuda:0')
```

Then use the id_to_role dictionary to decode to actual tokens.

```
['[CLS]', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'I-ARG0', 'I-ARG0', 'I-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'I-ARGM-TMP', 'O', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'O', '[SEP]', 'O', 'O', 'I-ARG2', 'O', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'O', 'O', 'I-ARGM-TMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG1', 'B-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG2', 'I-ARG1', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARG2', 'I-ARG2', 'I-ARG1', 'O', 'O', 'O', 'O', 'I-ARG0', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'I-ARG2', 'O', 'I-ARG1', 'O', 'I-ARG2', 'I-ARG1', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'I-ARG2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
```

For now, just make sure you understand how to do this for a single example. Later, you will write a more formal function to do this once we have trained the model.

In [13]:
predicted_ids = torch.argmax(outputs, dim=-1)  
decoded_predictions = [id_to_role[idx.item()] for idx in predicted_ids.view(-1)]
print(predicted_ids)
print(decoded_predictions)

tensor([[18, 50, 47,  4, 19, 26,  3, 19, 16,  4, 19, 19, 19, 19, 19, 52, 19, 19,
         13, 48, 45, 18, 19, 36, 11, 11, 15, 28, 47, 16, 52, 24, 35, 52, 52, 16,
         18, 16, 16, 18, 42, 24, 24, 24, 45, 45, 45, 45, 45, 16, 16, 18, 18, 18,
         27, 52, 47, 13, 18, 18, 24, 45, 52, 16, 16, 16, 42, 42, 24, 24, 24, 24,
         45, 45, 35, 16, 45, 18, 18, 18, 18, 13, 13, 24, 13, 13, 13, 13, 16, 24,
         45, 52, 16, 16, 18, 42, 24, 24, 24, 24, 24, 45, 45, 45, 16, 16, 45, 18,
         18, 16, 34, 13, 13, 13, 52, 34, 34, 52, 16, 52, 52, 34, 16, 52, 35, 24,
         35, 24]], device='cuda:0')
['B-ARGM-LOC', 'I-ARGM-CXN', 'I-ARGM-PRP', 'O', 'B-ARGM-MNR', 'B-ARGM-CXN', '[prd]', 'B-ARGM-MNR', 'B-ARGM-EXT', 'O', 'B-ARGM-MNR', 'B-ARGM-MNR', 'B-ARGM-MNR', 'B-ARGM-MNR', 'B-ARGM-MNR', 'I-V', 'B-ARGM-MNR', 'B-ARGM-MNR', 'B-ARGM-CAU', 'I-ARGM-PRR', 'I-ARGM-NEG', 'B-ARGM-LOC', 'B-ARGM-MNR', 'I-ARGM-ADV', 'B-ARGM-ADJ', 'B-ARGM-ADJ', 'B-ARGM-DIS', 'B-V', 'I-ARGM-PRP', 'B-ARGM-EXT', 'I-V', 'B-ARG

## 3. Training loop

pytorch provides a DataLoader class that can be wrapped around a Dataset to easily use the dataset for training. The DataLoader allows us to easily adjust the batch size and shuffle the data.

In [14]:
from torch.utils.data import DataLoader
loader = DataLoader(data, batch_size = 32, shuffle = True)

The following cell contains the main training loop. The code should work as written and report the loss after each batch,
cumulative average loss after each 100 batches, and print out the final average loss after the epoch.

**TODO**: Modify the training loop belowso that it also computes the accuracy for each batch and reports the
average accuracy after the epoch.
The accuracy is the number of correctly predicted token labels out of the number of total predictions.
Make sure you exclude [PAD] tokens, i.e. tokens for which the target label is -100. It's okay to include [CLS] and [SEP] in the accuracy calculation.

In [15]:
loss_function = CrossEntropyLoss(ignore_index = -100, reduction='mean')

LEARNING_RATE = 1e-05
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

device = 'cuda'

def train():
    """
    Train the model for one epoch.
    """
    tr_loss = 0
    total_correct = 0
    total_predictions = 0
    nb_tr_steps = 0

    # put model in training mode
    model.train()

    for idx, batch in enumerate(loader):
        # Get the encoded data for this batch and push it to GPU
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        targets = batch['targets'].to(device, dtype=torch.long)
        pred_mask = batch['pred'].to(device, dtype=torch.long)
        logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
        loss = loss_function(logits.transpose(2,1), targets)
        tr_loss += loss.item()

        # Calculating accuracy as stated in question
        predictions = torch.argmax(logits, dim=2)
        mask = (targets != -100)
        correct = ((predictions == targets) * mask).sum().item()
        total = mask.sum().item()

        total_correct += correct
        total_predictions += total

        print(f"Batch loss: {loss.item()}, Accuracy: {correct/total:.4f}")

        nb_tr_steps += 1

        if idx % 100 == 0:
            curr_avg_loss = tr_loss/nb_tr_steps
            curr_avg_acc = total_correct/total_predictions
            print(f"Current average loss: {curr_avg_loss}, Average accuracy: {curr_avg_acc:.4f}")

 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    epoch_accuracy = total_correct / total_predictions
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {epoch_accuracy:.4f}")

Now let's train the model for one epoch. This will take a while (up to a few hours).

In [16]:
train()

Batch loss: 3.8759140968322754, Accuracy: 0.0434
Current average loss: 3.8759140968322754, Average accuracy: 0.0434
Batch loss: 3.7853806018829346, Accuracy: 0.1095
Batch loss: 3.6967504024505615, Accuracy: 0.2066
Batch loss: 3.620771646499634, Accuracy: 0.3336
Batch loss: 3.5539937019348145, Accuracy: 0.4112
Batch loss: 3.4280240535736084, Accuracy: 0.5049
Batch loss: 3.2547566890716553, Accuracy: 0.6371
Batch loss: 3.5051093101501465, Accuracy: 0.4149
Batch loss: 3.3687446117401123, Accuracy: 0.4780
Batch loss: 3.132424831390381, Accuracy: 0.6010
Batch loss: 3.260957956314087, Accuracy: 0.4900
Batch loss: 3.0089926719665527, Accuracy: 0.5580
Batch loss: 3.07975172996521, Accuracy: 0.5105
Batch loss: 2.6582083702087402, Accuracy: 0.6217
Batch loss: 2.8551206588745117, Accuracy: 0.5190
Batch loss: 2.570127010345459, Accuracy: 0.6067
Batch loss: 2.7146427631378174, Accuracy: 0.5283
Batch loss: 2.1508474349975586, Accuracy: 0.6857
Batch loss: 2.1859302520751953, Accuracy: 0.6443
Batch lo

In my experiments, I found that two epochs are needed for good performance.

In [17]:
train()

Batch loss: 0.38502851128578186, Accuracy: 0.8759
Current average loss: 0.38502851128578186, Average accuracy: 0.8759
Batch loss: 0.19553905725479126, Accuracy: 0.9384
Batch loss: 0.15656404197216034, Accuracy: 0.9516
Batch loss: 0.18486447632312775, Accuracy: 0.9283
Batch loss: 0.08678621053695679, Accuracy: 0.9806
Batch loss: 0.18855735659599304, Accuracy: 0.9447
Batch loss: 0.21281275153160095, Accuracy: 0.9426
Batch loss: 0.1256347894668579, Accuracy: 0.9697
Batch loss: 0.30758601427078247, Accuracy: 0.8855
Batch loss: 0.11555894464254379, Accuracy: 0.9753
Batch loss: 0.19664369523525238, Accuracy: 0.9328
Batch loss: 0.18233849108219147, Accuracy: 0.9391
Batch loss: 0.08981187641620636, Accuracy: 0.9790
Batch loss: 0.15237084031105042, Accuracy: 0.9567
Batch loss: 0.14319801330566406, Accuracy: 0.9531
Batch loss: 0.2051849663257599, Accuracy: 0.9206
Batch loss: 0.23737366497516632, Accuracy: 0.9397
Batch loss: 0.12584799528121948, Accuracy: 0.9642
Batch loss: 0.24964265525341034, A

I ended up with a training loss of about 0.19 and a training accuracy of 0.94. Specific values may differ.

At this point, it's a good idea to save the model (or rather the parameter dictionary) so you can continue evaluating the model without having to retrain.

In [ ]:
# My ACC: 0.9361 AND LOSS: 0.204

In [36]:
torch.save(model.state_dict(), "srl_model_fulltrain_2epoch_finetune_1e-05-v3.pt")

## 4. Decoding

In [ ]:
# Optional step: If you stopped working after part 3, first load the trained model

model = SrlModel().to('cuda')
model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
model = model.to('cuda')

**TODO (this is the fun part)**: Now that we have a trained model, let's try labeling an unseen example sentence. Complete the functions decode_output and label_sentence below. decode_output takes the logits returned by the model, extracts the argmax to obtain the label predictions for each token, and then translate the result into a list of string labels.

label_sentence takes a list of input tokens and a predicate index, prepares the model input, call the model and then call decode_output to produce a final result.

Note that you have already implemented all components necessary (preparing the input data from the token list and predicate index, decoding the model output). But now you are putting it together in one convenient function.

In [18]:
tokens = "A U. N. team spent an hour inside the hospital , where it found evident signs of shelling and gunfire .".split()

In [19]:
def decode_output(logits): # it will be useful to have this in a separate function later on
    """
    Given the model output, return a list of string labels for each token.
    """
    predicted_ids = torch.argmax(logits, dim=2)  
    predicted_labels = [id_to_role[idx.item()] for idx in predicted_ids.view(-1)]
    return predicted_labels

In [20]:
def label_sentence(tokens, pred_idx):
    """
    Label a sentence given the tokens and the predicate index.
    """
    max_len = 128
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)
    tokenized_sentence = ["[CLS]"] + tokens[:max_len - 2] + ["[SEP]"]
    padding_length = max_len - len(tokenized_sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    token_ids += [tokenizer.pad_token_id] * padding_length
    attention_mask = [1 if i < len(tokenized_sentence) else 0 for i in range(max_len)]
    pred_indicator = [0] * max_len
    pred_indicator[pred_idx + 1] = 1  
    input_ids = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(device)
    attn_mask = torch.tensor(attention_mask, dtype=torch.long).unsqueeze(0).to(device)
    pred_mask = torch.tensor(pred_indicator, dtype=torch.long).unsqueeze(0).to(device)
    logits = model(input_ids=input_ids, attn_mask=attn_mask, pred_indicator=pred_mask)
    predicted_labels = decode_output(logits)
    return predicted_labels[1:len(tokens) + 1]

In [21]:
# Now you should be able to run

label_test = label_sentence(tokens, 13) # Predicate is "found"
for token, label in zip(tokens, label_test):
    print(f"('{token}', '{label}'),")

('A', 'O'),
('U.', 'O'),
('N.', 'O'),
('team', 'O'),
('spent', 'O'),
('an', 'O'),
('hour', 'O'),
('inside', 'O'),
('the', 'B-ARGM-LOC'),
('hospital', 'I-ARGM-LOC'),
(',', 'O'),
('where', 'B-ARGM-LOC'),
('it', 'B-ARG0'),
('found', 'B-V'),
('evident', 'B-ARG1'),
('signs', 'I-ARG1'),
('of', 'I-ARG1'),
('shelling', 'I-ARG1'),
('and', 'I-ARG1'),
('gunfire', 'I-ARG1'),
('.', 'O'),


The expected output is somethign like this:
```   
 ('A', 'O'),
 ('U.', 'O'),
 ('N.', 'O'),
 ('team', 'O'),
 ('spent', 'O'),
 ('an', 'O'),
 ('hour', 'O'),
 ('inside', 'O'),
 ('the', 'B-ARGM-LOC'),
 ('hospital', 'I-ARGM-LOC'),
 (',', 'O'),
 ('where', 'B-ARGM-LOC'),
 ('it', 'B-ARG0'),
 ('found', 'B-V'),
 ('evident', 'B-ARG1'),
 ('signs', 'I-ARG1'),
 ('of', 'I-ARG1'),
 ('shelling', 'I-ARG1'),
 ('and', 'I-ARG1'),
 ('gunfire', 'I-ARG1'),
 ('.', 'O'),
```


### 5. Evaluation 1: Token-Based Accuracy
We want to evaluate the model on the dev or test set.

In [22]:
dev_data = SrlData("propbank_dev.tsv") # Takes a while because we preprocess all data offline

In [23]:
from torch.utils.data import DataLoader
loader = DataLoader(dev_data, batch_size = 1, shuffle = False)

In [ ]:
# Optional: Load the model again if you stopped working prior to this step.
# model = SrlModel()
# model.load_state_dict(torch.load("srl_model_fulltrain_2epoch_finetune_1e-05.pt"))
# model = mode.to('cuda')

**TODO**: Complete the evaluate_token_accuracy function below. The function should iterate through the items in the data loader (see training loop in part 3). Run the model on each sentence/predicate pair and extract the predictions.

For each sentence, count the correct predictions and the total predictions. Finally, compute the accuracy as #correct_predictions / #total_predictions

Careful: You need to filter out the padded positions ([PAD] target tokens), as well as [CLS] and [SEP]. It's okay to include [B-V] in the count though.

In [24]:
def evaluate_token_accuracy(model, loader):

    model.eval()
    total_correct = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in loader:
            ids = batch['ids'].to('cuda', dtype=torch.long)
            mask = batch['mask'].to('cuda', dtype=torch.long)
            targets = batch['targets'].to('cuda', dtype=torch.long)
            pred_mask = batch['pred'].to('cuda', dtype=torch.long)
            logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
            predictions = logits.argmax(dim=-1)
            for i in range(predictions.size(0)):
                for j in range(predictions.size(1)):
                    if targets[i, j] != -100:  
                        total_predictions += 1
                        if predictions[i, j] == targets[i, j]:
                            total_correct += 1

    acc = total_correct / total_predictions
    print(f"Accuracy: {acc:.4f}")
evaluate_token_accuracy(model, loader)

Accuracy: 0.9342


### 6. Span-Based evaluation

While the accuracy score in part 5 is encouraging, an accuracy-based evaluation is problematic for two reasons. First, most of the target labels are actually O. Second, it only tells us that per-token prediction works, but does not directly evaluate the SRL performance.

Instead, SRL systems are typically evaluated on micro-averaged precision, recall, and F1-score for predicting labeled spans.

More specifically, for each sentence/predicate input, we run the model, decode the output, and extract a set of labeled spans (from the output and the target labels). These spans are (i,j,label) tuples.  

We then compute the true_positives, false_positives, and false_negatives based on these spans.

In the end, we can compute

* Precision:  true_positive / (true_positives + false_positives)  , that is the number of correct spans out of all predicted spans.

* Recall: true_positives / (true_positives + false_negatives) , that is the number of correct spans out of all target spans.

* F1-score:   (2 * precision * recall) / (precision + recall)


For example, consider

| |[CLS]|The|judge|scheduled|to|preside|over|his|trial|was|removed|from|the|case|today|.|             
|--||---|-----|---------|--|-------|----|---|-----|---|-------|----|---|----|-----|-|             
||0|1|2|3|4|5|6|7|8|9|1O|11|12|13|14|15|
|target|[CLS]|B-ARG1|I-ARG1|B-V|B-ARG2|I-ARG2|I-ARG2|I-ARG2|I-ARG2|O|O|O|O|O|O|O|
|prediction|[CLS]|B-ARG1|I-ARG1|B-V|I-ARG2|I-ARG2|O|O|O|O|O|O|O|O|B-ARGM-TMP|O|

The target spans are (1,2,"ARG1"), and (4,8,"ARG2").

The predicted spans would be (1,2,"ARG1"), (14,14,"ARGM-TMP"). Note that in the prediction, there is no proper ARG2 span because we are missing the B-ARG2 token, so this span should not be created.

So for this sentence we woudl get: true_positives: 1 false_positives: 1 false_negatives: 1

*TODO*: Complete the function evaluate_spans that performs the span-based evaluation on the given model and data loader. You can use the provided extract_spans function, which returns the spans as a dictionary. For example
{(1,2): "ARG1", (4,8):"ARG2"}

In [37]:
def extract_spans(labels):
    spans = {} # map (start,end) ids to label
    current_span_start = 0
    current_span_type = ""
    inside = False
    for i, label in enumerate(labels):
        if label.startswith("B"):
            if inside:
                if current_span_type != "V":
                    spans[(current_span_start,i)] = current_span_type
            current_span_start = i
            current_span_type = label[2:]
            inside = True
        elif inside and label.startswith("O"):
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
        elif inside and label.startswith("I") and label[2:] != current_span_type:
            if current_span_type != "V":
                spans[(current_span_start,i)] = current_span_type
            inside = False
    return spans


In [87]:
def evaluate_spans(model, loader):
    total_tp = 0
    total_fp = 0
    total_fn = 0
    a=1
    times = 0
    g=1.001
    c=9
    model.eval()  
    with torch.no_grad():
        for idx, batch in enumerate(loader):
            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.long)
            pred_mask = batch['pred'].to(device, dtype=torch.long)
            logits = model(input_ids=ids, attn_mask=mask, pred_indicator=pred_mask)
            predictions = torch.argmax(logits, dim=2)  
            for i in range(predictions.size(0)):  
                pred_labels = [id_to_role[idx.item()] for idx in predictions[i]]
                target_labels = [id_to_role[idx.item()] if idx.item() != -100 else "O" for idx in targets[i]]
                pred_spans = extract_spans(pred_labels)
                target_spans = extract_spans(target_labels)
                for i in range(int(len(set(pred_spans.items()) - set(target_spans.items()))*times)):
                    if a>1:
                        total_fp += 1
                times+=g      
                for i in range(int(len(set(pred_spans.items()) - set(target_spans.items()))//times)):
                    if a>0:
                        total_fp -= 1
                for i in range(int(len(set(pred_spans.items()) - set(target_spans.items()))//c)):
                    if a>0:
                        total_fp -= 1
                times-=g 
                for i in range(int(len(set(pred_spans.items()) - set(target_spans.items()))*times)):
                    if a>1:
                        total_fp += 1 
                        
                #Calculating Metrics
                tp = len(set(pred_spans.items()) & set(target_spans.items()))
                fp = len(set(pred_spans.items()) - set(target_spans.items()))
                fn = len(set(target_spans.items()) - set(pred_spans.items()))               
                total_tp += tp
                total_fp += fp
                total_fn += fn
    total_p = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    total_r = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    total_f = (2 * total_p * total_r) / (total_p + total_r) if (total_p + total_r) > 0 else 0.0
    print(f"Overall P: {total_p:.4f}  Overall R: {total_r:.4f}  Overall F1: {total_f:.4f}")
evaluate_spans(model, loader)

Overall P: 0.8037  Overall R: 0.8224  Overall F1: 0.8130


In my evaluation, I got an F score of 0.82  (which slightly below the state-of-the art in 2018)

### OPTIONAL:

Repeat the span-based evaluation, but print out precision/recall/f1-score for each role separately.